In [ ]:
# Example 3: Analyze specific game performance
print("=== EXAMPLE: WEEK 1 RAVENS @ CHIEFS GAME ANALYSIS ===")

# Analyze the season opener
game_analysis = analyze_game_performance('2024_001', tables, schedule_df)

if 'error' not in game_analysis:
    game_info = game_analysis['game_info']
    print(f"Game: {game_info['away_team']} @ {game_info['home_team']}")
    print(f"Date: {game_info['game_date']}")
    print(f"Week: {game_info['week']}")
    print(f"TV: {game_info['tv_network']}")
    
    print(f"\nPlayer Performances:")
    for table_name, players in game_analysis['player_performances'].items():
        print(f"\n{table_name.upper()} ({len(players)} players):")
        for player in players[:3]:  # Show top 3 players per table
            print(f"  {player.get('Player', 'Unknown')} ({player.get('Tm', 'Unknown')})")
            
    print(f"\nTeam Totals:")
    for table_name, team_stats in game_analysis['team_totals'].items():
        print(f"\n{table_name.upper()}:")
        for team, stats in team_stats.items():
            key_stats = {k: v for k, v in stats.items() if k in ['Pass_Yds', 'Rush_Yds', 'Pass_TD', 'Rush_TD'] and v > 0}
            if key_stats:
                print(f"  {team}: {key_stats}")
else:
    print(f"Error: {game_analysis['error']}")

In [ ]:
# Example 2: Create detailed player profile
print("=== EXAMPLE: LAMAR JACKSON PLAYER PROFILE ===")

# Create comprehensive profile for a specific player
lamar_profile = create_player_profile("Lamar Jackson", tables, schedule_df)

print(f"Player: {lamar_profile['player_name']}")
print(f"Season: {lamar_profile['season']}")
print(f"Games Played: {lamar_profile['summary']['games_played']}")
print(f"Teams: {', '.join(lamar_profile['summary']['teams'])}")
print(f"Weeks Active: {lamar_profile['summary']['weeks_active']}")

print(f"\nGame Log (first 5 games):")
for i, game in enumerate(lamar_profile['game_log'][:5]):
    print(f"  Week {game['week']}: {game['team']} vs {game['opponent']} ({game['home_away']})")
    print(f"    Table: {game['table_type']}")
    if game['stats']:
        key_stats = {k: v for k, v in game['stats'].items() if k in ['Pass_Yds', 'Rush_Yds', 'Pass_TD', 'Rush_TD']}
        print(f"    Stats: {key_stats}")
    print()

In [ ]:
# Example 1: Get top rushing yards leaders
print("=== TOP 10 RUSHING YARDS LEADERS ===")
top_rushers = get_top_players_by_stat('Rush_Yds', 'basic_offense', tables, limit=10)
if not top_rushers.empty:
    print(top_rushers[['Player', 'Rush_Yds', 'games_played', 'per_game']].to_string(index=False))
else:
    print("No rushing data found")

print("\n=== TOP 10 PASSING YARDS LEADERS ===")
top_passers = get_top_players_by_stat('Pass_Yds', 'basic_offense', tables, limit=10)
if not top_passers.empty:
    print(top_passers[['Player', 'Pass_Yds', 'games_played', 'per_game']].to_string(index=False))

print("\n=== TOP 10 RECEIVING YARDS LEADERS ===")
top_receivers = get_top_players_by_stat('Rec_Yds', 'basic_offense', tables, limit=10)
if not top_receivers.empty:
    print(top_receivers[['Player', 'Rec_Yds', 'games_played', 'per_game']].to_string(index=False))

# Example Usage - Player Analysis Interface

Demonstrate the player aggregation functions with example queries.

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

def load_all_game_data():
    """Load all 4 statistical tables into a dictionary"""
    data_dir = "nfl_2024_data"
    tables = {}
    
    table_files = {
        'basic_offense': f'{data_dir}/2024_season_basic_offense.csv',
        'advanced_passing': f'{data_dir}/2024_season_advanced_passing.csv',
        'advanced_rushing': f'{data_dir}/2024_season_advanced_rushing.csv',
        'advanced_receiving': f'{data_dir}/2024_season_advanced_receiving.csv'
    }
    
    for table_name, filepath in table_files.items():
        if os.path.exists(filepath):
            tables[table_name] = pd.read_csv(filepath)
            print(f"Loaded {table_name}: {len(tables[table_name])} records")
        else:
            print(f"WARNING: {filepath} not found")
            tables[table_name] = pd.DataFrame()
    
    return tables

def get_player_season_stats(player_name, tables):
    """Get complete season statistics for a specific player"""
    
    player_stats = {
        'player_name': player_name,
        'games_played': 0,
        'teams': set(),
        'positions': set(),
        'game_log': [],
        'season_totals': {},
        'averages': {},
        'advanced_metrics': {}
    }
    
    # Process each table type
    for table_name, df in tables.items():
        if df.empty:
            continue
            
        # Find player records
        player_games = df[df['Player'].str.contains(player_name, case=False, na=False)]
        
        if not player_games.empty:
            player_stats['games_played'] = max(player_stats['games_played'], len(player_games))
            
            # Collect team and position info
            if 'Tm' in player_games.columns:
                player_stats['teams'].update(player_games['Tm'].dropna().unique())
            
            # Add game-by-game data
            for _, game in player_games.iterrows():
                game_data = {
                    'game_id': game.get('game_id', 'Unknown'),
                    'table_type': table_name,
                    'team': game.get('Tm', 'Unknown'),
                    'stats': game.to_dict()
                }
                player_stats['game_log'].append(game_data)
    
    # Convert sets to lists for JSON serialization
    player_stats['teams'] = list(player_stats['teams'])
    player_stats['positions'] = list(player_stats['positions'])
    
    return player_stats

def create_player_profile(player_name, tables, schedule_df=None):
    """Create comprehensive player profile with game context"""
    
    # Load schedule data if not provided
    if schedule_df is None:
        schedule_df = pd.read_csv('nfl_2024_schedule.csv')
    
    profile = {
        'player_name': player_name,
        'season': 2024,
        'last_updated': datetime.now().isoformat(),
        'summary': {},
        'game_log': [],
        'season_totals': {},
        'averages': {}
    }
    
    # Collect all game appearances
    all_games = []
    
    for table_name, df in tables.items():
        if df.empty:
            continue
            
        player_games = df[df['Player'].str.contains(player_name, case=False, na=False)]
        
        for _, game in player_games.iterrows():
            game_id = game.get('game_id', 'Unknown')
            
            # Get game context from schedule
            game_info = schedule_df[schedule_df['game_id'] == game_id]
            
            game_record = {
                'game_id': game_id,
                'table_type': table_name,
                'team': game.get('Tm', 'Unknown'),
                'week': game_info['week'].iloc[0] if not game_info.empty else None,
                'date': game_info['game_date'].iloc[0] if not game_info.empty else None,
                'opponent': None,
                'home_away': None,
                'stats': {k: v for k, v in game.to_dict().items() if k not in ['game_id', 'Player', 'Tm']}
            }
            
            # Determine opponent and home/away status
            if not game_info.empty:
                row = game_info.iloc[0]
                player_team = game.get('Tm', 'Unknown')
                
                if player_team == row['home_team']:
                    game_record['opponent'] = row['away_team']
                    game_record['home_away'] = 'home'
                elif player_team == row['away_team']:
                    game_record['opponent'] = row['home_team'] 
                    game_record['home_away'] = 'away'
            
            all_games.append(game_record)
    
    # Sort games by week
    all_games.sort(key=lambda x: x.get('week', 999))
    profile['game_log'] = all_games
    
    # Calculate summary stats
    profile['summary']['games_played'] = len(set(game['game_id'] for game in all_games))
    profile['summary']['teams'] = list(set(game['team'] for game in all_games))
    profile['summary']['weeks_active'] = len(set(game['week'] for game in all_games if game['week']))
    
    return profile

def get_top_players_by_stat(stat_column, table_name, tables, limit=10):
    """Get top players by a specific statistic"""
    
    if table_name not in tables or tables[table_name].empty:
        return pd.DataFrame()
    
    df = tables[table_name]
    
    if stat_column not in df.columns:
        return pd.DataFrame()
    
    # Convert to numeric and aggregate by player
    df_numeric = df.copy()
    df_numeric[stat_column] = pd.to_numeric(df_numeric[stat_column], errors='coerce')
    
    # Group by player and sum the statistic
    player_totals = df_numeric.groupby('Player')[stat_column].sum().reset_index()
    player_totals = player_totals.sort_values(stat_column, ascending=False).head(limit)
    
    # Add games played
    games_played = df_numeric.groupby('Player').size().reset_index(name='games_played')
    player_totals = player_totals.merge(games_played, on='Player')
    
    # Calculate per-game average
    player_totals['per_game'] = player_totals[stat_column] / player_totals['games_played']
    
    return player_totals

def analyze_game_performance(game_id, tables, schedule_df=None):
    """Analyze all player performances in a specific game"""
    
    if schedule_df is None:
        schedule_df = pd.read_csv('nfl_2024_schedule.csv')
    
    # Get game info
    game_info = schedule_df[schedule_df['game_id'] == game_id]
    
    if game_info.empty:
        return {'error': f'Game {game_id} not found'}
    
    game_context = game_info.iloc[0].to_dict()
    
    analysis = {
        'game_id': game_id,
        'game_info': game_context,
        'player_performances': {},
        'team_totals': {},
        'top_performers': {}
    }
    
    # Collect all player data for this game
    for table_name, df in tables.items():
        if df.empty:
            continue
            
        game_players = df[df['game_id'] == game_id]
        
        if not game_players.empty:
            analysis['player_performances'][table_name] = game_players.to_dict('records')
            
            # Calculate team totals for numeric columns
            numeric_cols = game_players.select_dtypes(include=[np.number]).columns
            team_totals = {}
            
            for team in game_players['Tm'].unique():
                if pd.notna(team):
                    team_data = game_players[game_players['Tm'] == team]
                    team_totals[team] = team_data[numeric_cols].sum().to_dict()
            
            analysis['team_totals'][table_name] = team_totals
    
    return analysis

# Load all data for analysis
print("Loading NFL 2024 player data...")
tables = load_all_game_data()

# Load schedule for game context
schedule_df = pd.read_csv('nfl_2024_schedule.csv')
print(f"Loaded schedule: {len(schedule_df)} games")

print("\nPlayer aggregation functions ready!")

# Player Aggregation Functions

Create comprehensive player profiles and season statistics from game-level data.

In [ ]:
# Core imports for NFL schedule parsing
import re
import pandas as pd
import json
import pdfplumber
from datetime import datetime

In [ ]:
# DEPRECATED: Old PyPDF2 parser - replaced by pdfplumber version
# This function had issues with mangled text extraction and incorrect dates
# Left here for reference but use parse_nfl_with_pdfplumber_full instead

In [ ]:
import pdfplumber
import pandas as pd
import re
import json
from datetime import datetime

def parse_nfl_with_pdfplumber_full(pdf_path, season_year):
    """Parse NFL schedule PDF using pdfplumber - full version with URL generation"""
    
    # Load team codes for URL generation
    with open('teams.json', 'r') as f:
        team_codes = json.load(f)
    
    games = []
    game_id = 1
    current_week = 1
    current_date = None
    
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            # Extract text from page
            text = page.extract_text()
            if not text:
                continue
                
            lines = text.split('\n')
            
            for line in lines:
                line = line.strip()
                if not line:
                    continue
                
                # Check for week headers
                if 'WEEK' in line and re.search(r'WEEK\s+\d+', line):
                    week_match = re.search(r'WEEK\s+(\d+)', line)
                    if week_match:
                        current_week = int(week_match.group(1))
                
                # Check for date patterns
                date_match = re.search(r'(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+(January|February|March|April|May|June|July|August|September|October|November|December)\s+(\d{1,2}),?\s*(\d{4})', line)
                if date_match:
                    day_name = date_match.group(1)
                    month_name = date_match.group(2)
                    day = int(date_match.group(3))
                    year = int(date_match.group(4))
                    
                    month_map = {
                        'January': 1, 'February': 2, 'March': 3, 'April': 4,
                        'May': 5, 'June': 6, 'July': 7, 'August': 8,
                        'September': 9, 'October': 10, 'November': 11, 'December': 12
                    }
                    month = month_map[month_name]
                    current_date = datetime(year, month, day).date()
                
                # Check for games - both "at" and "vs" (international)
                elif (' at ' in line or ' vs ' in line) and current_date:
                    # Skip header lines
                    if 'GAME' in line and 'LOCAL' in line and 'ET' in line:
                        continue
                    
                    # Split on either "at" or "vs"
                    if ' at ' in line:
                        parts = line.split(' at ')
                        is_international = False
                    else:
                        parts = line.split(' vs ')
                        is_international = True
                        
                    if len(parts) == 2:
                        away_team = parts[0].strip()
                        
                        # Extract home team (before times/other info)
                        rest = parts[1]
                        # Split on common patterns to isolate team name
                        home_parts = re.split(r'\s+\d+:\d+|\s+\([^)]*\)\s+\d|\s+[A-Z]{2,4}\s*$', rest)
                        home_team = home_parts[0].strip()
                        
                        # Clean up team names - REMOVE TBD suffixes
                        away_team = re.sub(r'\s+TBD.*$', '', away_team).strip()
                        home_team = re.sub(r'\s+TBD.*$', '', home_team).strip()
                        home_team = re.sub(r'\s+\([^)]*\)$', '', home_team)  # Remove trailing parentheses
                        
                        if away_team and home_team and len(away_team) > 3 and len(home_team) > 3:
                            # Generate PFR URL
                            home_code = team_codes.get(home_team)
                            away_code = team_codes.get(away_team)
                            
                            pfr_url = None
                            if home_code and current_date:
                                date_str = current_date.strftime('%Y%m%d')
                                pfr_url = f"https://www.pro-football-reference.com/boxscores/{date_str}0{home_code}.htm"
                            
                            # Extract additional details
                            day_match = re.search(r'\((Thu|Mon|Tue|Wed|Fri|Sat|Sun)\)', line)
                            day_of_week = day_match.group(1) if day_match else None
                            
                            times = re.findall(r'\d+:\d+p', line)
                            local_time = times[0] if times else None
                            et_time = times[1] if len(times) > 1 else times[0] if times else None
                            
                            networks = ['NBC', 'CBS', 'FOX', 'ESPN', 'NFLN', 'Prime Video', 'Peacock', 'ESPN/ABC']
                            tv_network = None
                            for network in networks:
                                if network in line:
                                    tv_network = network
                                    break
                            
                            games.append({
                                'game_id': f"{season_year}_{game_id:03d}",
                                'season': season_year,
                                'week': current_week,
                                'game_date': current_date,
                                'day_of_week': day_of_week,
                                'away_team': away_team,
                                'home_team': home_team,
                                'local_time': local_time,
                                'et_time': et_time,
                                'tv_network': tv_network,
                                'pfr_url': pfr_url,
                                'pfr_home_code': home_code,
                                'pfr_away_code': away_code,
                                'is_international': is_international
                            })
                            game_id += 1
    
    return pd.DataFrame(games)

# Parse the full 2024 season
print("Parsing full 2024 NFL schedule with pdfplumber...")
games_2024_full = parse_nfl_with_pdfplumber_full('NFL-Regular-season-2024.pdf', 2024)

print(f"Found {len(games_2024_full)} games total")

# Check for international games
international_games = games_2024_full[games_2024_full['is_international'] == True]
print(f"International games (vs): {len(international_games)}")

# Check for any remaining URL issues
missing_urls = games_2024_full[games_2024_full['pfr_url'].isna()]
print(f"Games with missing URLs: {len(missing_urls)}")

if len(missing_urls) > 0:
    print("Games still missing URLs:")
    for _, game in missing_urls.iterrows():
        print(f"  {game['game_id']}: {game['away_team']} @ {game['home_team']}")

print("\nFirst 5 games:")
print(games_2024_full[['game_id', 'week', 'game_date', 'away_team', 'home_team', 'is_international']].head(5))

print("\nLast 5 games:")
print(games_2024_full[['game_id', 'week', 'game_date', 'away_team', 'home_team', 'is_international']].tail(5))

# Save to CSV for batch scraper
csv_filename = 'nfl_2024_schedule.csv'
games_2024_full.to_csv(csv_filename, index=False)
print(f"\n📄 Saved {len(games_2024_full)} games to {csv_filename} for batch scraper")

# Also save corrected version for auditing
games_2024_full.to_csv('nfl_2024_schedule_corrected.csv', index=False)
print(f"📄 Saved audit copy to nfl_2024_schedule_corrected.csv")

# NFL Player Stats Scraper

Test scraping player statistics from Pro Football Reference box scores.

In [ ]:
# REMOVED: Debug code for testing single game scraping
# This was used during development but no longer needed

In [ ]:
# REMOVED: Debug code for parsing table structure
# This was used during development but no longer needed

In [ ]:
# REMOVED: Debug code for finding advanced tables in HTML comments
# This was used during development but no longer needed

In [ ]:
import re
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from datetime import datetime

def datafix_failed_games():
    """Scrape failed games and append to existing data files"""
    
    print("=== NFL DATAFIX SCRIPT ===")
    
    # Manual corrections for rescheduled games
    manual_fixes = {
        '2024_225': {  # Browns @ Bengals - moved TO 12/22
            'correct_date': '2024-12-22',
            'correct_url': 'https://www.pro-football-reference.com/boxscores/202412220cin.htm'
        },
        '2024_235': {  # Broncos @ Chargers - moved TO 12/19
            'correct_date': '2024-12-19',
            'correct_url': 'https://www.pro-football-reference.com/boxscores/202412190sdg.htm'
        },
        '2024_247': {  # Colts @ Giants
            'correct_date': '2024-12-29',
            'correct_url': 'https://www.pro-football-reference.com/boxscores/202412290nyg.htm'
        },
        '2024_248': {  # Falcons @ Commanders  
            'correct_date': '2024-12-29',
            'correct_url': 'https://www.pro-football-reference.com/boxscores/202412290was.htm'
        },
        '2024_270': {  # Bengals @ Steelers
            'correct_date': '2025-01-04', 
            'correct_url': 'https://www.pro-football-reference.com/boxscores/202501040pit.htm'
        }
    }
    
    # Read error log to get failed game IDs
    with open('scraper_errors.log', 'r') as f:
        error_lines = f.readlines()
    
    failed_game_ids = []
    for line in error_lines:
        match = re.search(r'(2024_\d+)', line)
        if match:
            game_id = match.group(1)
            if game_id not in failed_game_ids:
                failed_game_ids.append(game_id)
    
    print(f"Found {len(failed_game_ids)} failed games to retry")
    print(f"Manual fixes available for: {list(manual_fixes.keys())}")
    
    # Load updated schedule
    schedule_df = pd.read_csv('nfl_2024_schedule.csv')
    failed_games = schedule_df[schedule_df['game_id'].isin(failed_game_ids)].copy()
    
    # Apply manual fixes
    for idx, row in failed_games.iterrows():
        if row['game_id'] in manual_fixes:
            fix = manual_fixes[row['game_id']]
            failed_games.at[idx, 'game_date'] = fix['correct_date']
            failed_games.at[idx, 'pfr_url'] = fix['correct_url']
            print(f"Applied manual fix for {row['game_id']}: {fix['correct_url']}")
    
    # Skip future games (not played yet)
    today = datetime.now().date()
    playable_games = []
    
    for _, game in failed_games.iterrows():
        game_date = datetime.strptime(game['game_date'], '%Y-%m-%d').date()
        if game_date <= today:
            playable_games.append(game)
    
    print(f"Games that can be scraped (already played): {len(playable_games)}")
    
    if not playable_games:
        print("No games to scrape - all failed games are in the future")
        return
    
    # Load existing data files to append to
    output_dir = "nfl_2024_data"
    existing_tables = {}
    table_files = {
        'basic_offense': f'{output_dir}/2024_season_basic_offense.csv',
        'advanced_passing': f'{output_dir}/2024_season_advanced_passing.csv',
        'advanced_rushing': f'{output_dir}/2024_season_advanced_rushing.csv',
        'advanced_receiving': f'{output_dir}/2024_season_advanced_receiving.csv'
    }
    
    for table_name, filepath in table_files.items():
        try:
            existing_tables[table_name] = pd.read_csv(filepath)
            print(f"Loaded {table_name}: {len(existing_tables[table_name])} existing records")
        except FileNotFoundError:
            print(f"WARNING: {filepath} not found - will create new file")
            existing_tables[table_name] = pd.DataFrame()
    
    # Import the scraper function
    from bs4 import Comment
    
    def scrape_game_tables_with_id(soup, game_id, game_url):
        """Extract all 4 tables and add game_id to each record"""
        tables = {}
        
        # 1. Basic offense table  
        basic_table = soup.find('table', {'id': 'player_offense'})
        if basic_table:
            basic_data = []
            tbody = basic_table.find('tbody')
            if tbody:
                rows = tbody.find_all('tr')
                for row in rows:
                    if row.get('class') and ('thead' in str(row.get('class')) or 'over_header' in str(row.get('class'))):
                        continue
                    cells = [td.get_text().strip() for td in row.find_all(['td', 'th'])]
                    if cells and cells[0] == 'Player':
                        continue
                    if len(cells) > 1 and cells[0]:
                        row_data = [game_id] + cells
                        basic_data.append(row_data)
            
            if basic_data:
                columns = ['game_id', 'Player', 'Tm', 'Pass_Cmp', 'Pass_Att', 'Pass_Yds', 'Pass_TD', 'Pass_Int', 
                          'Pass_Sk', 'Pass_Sk_Yds', 'Pass_Lng', 'Pass_Rate', 'Rush_Att', 'Rush_Yds', 
                          'Rush_TD', 'Rush_Lng', 'Rec_Tgt', 'Rec_Rec', 'Rec_Yds', 'Rec_TD', 'Rec_Lng', 
                          'Fmb', 'FL']
                while len(columns) < len(basic_data[0]):
                    columns.append(f'Col_{len(columns)}')
                tables['basic_offense'] = pd.DataFrame(basic_data, columns=columns[:len(basic_data[0])])
        
        # 2-4. Advanced tables from comments
        comments = soup.find_all(string=lambda text: isinstance(text, Comment))
        advanced_sections = {
            'advanced_passing': 'Advanced Passing',
            'advanced_rushing': 'Advanced Rushing', 
            'advanced_receiving': 'Advanced Receiving'
        }
        
        for table_name, section_text in advanced_sections.items():
            for comment in comments:
                if section_text.lower() in comment.lower():
                    comment_soup = BeautifulSoup(comment, 'html.parser')
                    adv_table = comment_soup.find('table')
                    
                    if adv_table:
                        headers = ['game_id']
                        header_row = adv_table.find('thead') or adv_table.find('tr')
                        if header_row:
                            for th in header_row.find_all(['th', 'td']):
                                headers.append(th.get_text().strip())
                        
                        data_rows = []
                        rows = adv_table.find_all('tr')[1:]
                        for row in rows:
                            if row.get('class') and ('thead' in str(row.get('class')) or 'over_header' in str(row.get('class'))):
                                continue
                            cells = [td.get_text().strip() for td in row.find_all(['td', 'th'])]
                            if cells and cells[0] == 'Player':
                                continue
                            if len(cells) > 1 and cells[0]:
                                row_data = [game_id] + cells
                                data_rows.append(row_data)
                        
                        if data_rows and headers:
                            max_cols = max(len(headers), len(data_rows[0]) if data_rows else 0)
                            while len(headers) < max_cols:
                                headers.append(f'Col_{len(headers)}')
                            tables[table_name] = pd.DataFrame(data_rows, columns=headers[:len(data_rows[0])])
                    break
        
        return tables
    
    # Scrape the failed games
    scraped_count = 0
    for game in playable_games:
        game_id = game['game_id']
        url = game['pfr_url']
        
        print(f"\nScraping {game_id}: {game['away_team']} @ {game['home_team']}")
        print(f"URL: {url}")
        
        try:
            response = requests.get(url)
            
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                game_tables = scrape_game_tables_with_id(soup, game_id, url)
                
                # Check if any data was found
                total_records = sum(len(df) for df in game_tables.values() if not df.empty)
                
                if total_records == 0:
                    print(f"  WARNING: No data found (game may not have occurred)")
                else:
                    # Append to existing tables
                    for table_name, new_data in game_tables.items():
                        if not new_data.empty:
                            existing_tables[table_name] = pd.concat([existing_tables[table_name], new_data], ignore_index=True)
                            print(f"  Added {len(new_data)} records to {table_name}")
                
                scraped_count += 1
                print(f"  SUCCESS")
                
            else:
                print(f"  ERROR: HTTP {response.status_code}")
                
        except Exception as e:
            print(f"  ERROR: {e}")
        
        # Be respectful with delays
        time.sleep(2)
    
    # Save updated files
    print(f"\n=== SAVING UPDATED FILES ===")
    for table_name, df in existing_tables.items():
        if not df.empty:
            filepath = table_files[table_name]
            df.to_csv(filepath, index=False)
            print(f"Saved {table_name}: {len(df)} total records -> {filepath}")
    
    print(f"\nDatafix complete: {scraped_count} games processed")

# Datafix function preserved for future use
# Run with: datafix_failed_games()

# Batch NFL Stats Scraper

Scrape all 272 games from 2024 season in random order with delays.

In [ ]:
import random
import time
import os
import shutil
from datetime import datetime, timedelta

def scrape_game_tables_with_id(soup, game_id, game_url):
    """Extract all 4 tables and add game_id to each record"""
    
    tables = {}
    
    # 1. Basic offense table  
    basic_table = soup.find('table', {'id': 'player_offense'})
    if basic_table:
        basic_data = []
        tbody = basic_table.find('tbody')
        if tbody:
            rows = tbody.find_all('tr')
            
            for row in rows:
                if row.get('class') and ('thead' in str(row.get('class')) or 'over_header' in str(row.get('class'))):
                    continue
                    
                cells = [td.get_text().strip() for td in row.find_all(['td', 'th'])]
                
                if cells and cells[0] == 'Player':
                    continue
                    
                if len(cells) > 1 and cells[0]:
                    # Add game_id as first column
                    row_data = [game_id] + cells
                    basic_data.append(row_data)
        
        if basic_data:
            columns = ['game_id', 'Player', 'Tm', 'Pass_Cmp', 'Pass_Att', 'Pass_Yds', 'Pass_TD', 'Pass_Int', 
                      'Pass_Sk', 'Pass_Sk_Yds', 'Pass_Lng', 'Pass_Rate', 'Rush_Att', 'Rush_Yds', 
                      'Rush_TD', 'Rush_Lng', 'Rec_Tgt', 'Rec_Rec', 'Rec_Yds', 'Rec_TD', 'Rec_Lng', 
                      'Fmb', 'FL']
            while len(columns) < len(basic_data[0]):
                columns.append(f'Col_{len(columns)}')
            
            tables['basic_offense'] = pd.DataFrame(basic_data, columns=columns[:len(basic_data[0])])
    
    # 2-4. Advanced tables from comments
    from bs4 import Comment
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))
    
    advanced_sections = {
        'advanced_passing': 'Advanced Passing',
        'advanced_rushing': 'Advanced Rushing', 
        'advanced_receiving': 'Advanced Receiving'
    }
    
    for table_name, section_text in advanced_sections.items():
        for comment in comments:
            if section_text.lower() in comment.lower():
                comment_soup = BeautifulSoup(comment, 'html.parser')
                adv_table = comment_soup.find('table')
                
                if adv_table:
                    headers = ['game_id']  # Start with game_id
                    header_row = adv_table.find('thead') or adv_table.find('tr')
                    if header_row:
                        for th in header_row.find_all(['th', 'td']):
                            headers.append(th.get_text().strip())
                    
                    data_rows = []
                    rows = adv_table.find_all('tr')[1:]
                    
                    for row in rows:
                        if row.get('class') and ('thead' in str(row.get('class')) or 'over_header' in str(row.get('class'))):
                            continue
                            
                        cells = [td.get_text().strip() for td in row.find_all(['td', 'th'])]
                        
                        if cells and cells[0] == 'Player':
                            continue
                            
                        if len(cells) > 1 and cells[0]:
                            # Add game_id as first column
                            row_data = [game_id] + cells
                            data_rows.append(row_data)
                    
                    if data_rows and headers:
                        max_cols = max(len(headers), len(data_rows[0]) if data_rows else 0)
                        while len(headers) < max_cols:
                            headers.append(f'Col_{len(headers)}')
                        
                        tables[table_name] = pd.DataFrame(data_rows, columns=headers[:len(data_rows[0])])
                break
    
    return tables

def log_error(message, log_file="scraper_errors.log"):
    """Log errors to file with timestamp"""
    with open(log_file, 'a') as f:
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        f.write(f"[{timestamp}] {message}\n")

def format_time_remaining(seconds):
    """Format seconds into readable time string"""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    if hours > 0:
        return f"{hours}h {minutes}m"
    else:
        return f"{minutes}m"

def load_nfl_facts():
    """Load NFL facts from facts.txt file"""
    try:
        with open('facts.txt', 'r') as f:
            content = f.read()
        
        # Split by lines and filter out empty lines
        facts = [line.strip() for line in content.split('\n') if line.strip() and not line.strip().isdigit()]
        return facts
    except FileNotFoundError:
        return ["Did you know? The NFL was founded in 1920!"]  # Fallback fact

def batch_scrape_season():
    """
    WARNING: This will scrape ALL 272 games from 2024 season!
    Estimated time: 2-3 hours with 15-45 second delays
    """
    
    print("BATCH NFL SCRAPER STARTING")
    print("This will take 2-3 hours to complete all 272 games")
    
    # Clean up previous runs
    files_to_remove = ["scraper_errors.log"] + [f for f in os.listdir('.') if f.startswith('clean_game_')]
    for file in files_to_remove:
        if os.path.exists(file):
            os.remove(file)
    
    if os.path.exists("nfl_2024_data"):
        shutil.rmtree("nfl_2024_data")
        print("Cleaned up previous run data")
    
    # Load NFL facts for entertainment
    nfl_facts = load_nfl_facts()
    
    # Create output directory
    output_dir = "nfl_2024_data"
    os.makedirs(output_dir)
    print(f"Created output directory: {output_dir}")
    
    # Initialize error log
    log_error("=== BATCH SCRAPING SESSION STARTED ===")
    
    start_time = datetime.now()
    print(f"\nSCRAPING STARTED at {start_time.strftime('%H:%M:%S')}")
    
    # Load schedule
    schedule_df = pd.read_csv('nfl_2024_schedule.csv')
    total_games = len(schedule_df)
    print(f"Loaded {total_games} games to scrape")
    
    # Calculate time estimates
    avg_delay = (15 + 45) / 2  # 30 seconds average
    estimated_total_seconds = total_games * avg_delay
    estimated_hours = estimated_total_seconds / 3600
    estimated_end = start_time + timedelta(seconds=estimated_total_seconds)
    
    print(f"Estimated completion: {estimated_end.strftime('%H:%M:%S')} ({estimated_hours:.1f} hours)")
    
    # Randomize order
    games_to_scrape = schedule_df.sample(frac=1).reset_index(drop=True)
    print("Shuffled games randomly")
    
    # Initialize master DataFrames
    master_tables = {
        'basic_offense': [],
        'advanced_passing': [],
        'advanced_rushing': [],
        'advanced_receiving': []
    }
    
    successful_scrapes = 0
    failed_scrapes = 0
    
    for i, game in games_to_scrape.iterrows():
        game_id = game['game_id']
        url = game['pfr_url']
        
        # Progress calculation
        progress_pct = (i / total_games) * 100
        games_remaining = total_games - i - 1
        elapsed = datetime.now() - start_time
        
        print(f"\n[{i+1}/{total_games}] ({progress_pct:.1f}%) Scraping {game_id}")
        print(f"Game: {game['away_team']} @ {game['home_team']}")
        print(f"URL: {url}")
        
        try:
            # Make request
            response = requests.get(url)
            
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                game_tables = scrape_game_tables_with_id(soup, game_id, url)
                
                # Check if we got any data
                total_players = sum(len(df) for df in game_tables.values() if not df.empty)
                
                if total_players == 0:
                    # No data found - log as potential date/parsing issue
                    error_msg = f"NO DATA FOUND - {game_id} | {url} | Teams: {game['away_team']} @ {game['home_team']} | Date: {game.get('game_date', 'Unknown')}"
                    log_error(error_msg)
                    print(f"  WARNING: No player data found - logged to errors")
                else:
                    # Accumulate results
                    table_counts = {}
                    for table_name, df in game_tables.items():
                        if not df.empty:
                            master_tables[table_name].append(df)
                            table_counts[table_name] = len(df)
                        else:
                            table_counts[table_name] = 0
                    
                    # Display table counts - more concise
                    total_records = sum(table_counts.values())
                    print(f"  Data: {total_records} total records ({', '.join([f'{k}: {v}' for k, v in table_counts.items()])})")
                
                successful_scrapes += 1
                success_rate = (successful_scrapes / (successful_scrapes + failed_scrapes)) * 100
                print(f"  SUCCESS ({successful_scrapes}/{successful_scrapes+failed_scrapes}) {success_rate:.1f}%")
                
            elif response.status_code == 404:
                # Try alternate URL with other team's code
                print(f"  404 on primary URL, trying alternate...")
                
                # Create alternate URL with away team code - handle both string and datetime
                if isinstance(game['game_date'], str):
                    date_str = game['game_date'].replace('-', '')  # Convert 2024-12-08 to 20241208
                else:
                    date_str = game['game_date'].strftime('%Y%m%d') if pd.notnull(game.get('game_date')) else 'UNKNOWN'
                
                alt_url = f"https://www.pro-football-reference.com/boxscores/{date_str}0{game['pfr_away_code']}.htm"
                print(f"  Alt URL: {alt_url}")
                
                alt_response = requests.get(alt_url)
                
                if alt_response.status_code == 200:
                    print(f"  SUCCESS with alternate URL")
                    soup = BeautifulSoup(alt_response.content, 'html.parser')
                    game_tables = scrape_game_tables_with_id(soup, game_id, alt_url)
                    
                    # Check if we got any data
                    total_players = sum(len(df) for df in game_tables.values() if not df.empty)
                    
                    if total_players == 0:
                        error_msg = f"NO DATA FOUND - {game_id} | {alt_url} | Teams: {game['away_team']} @ {game['home_team']} | Date: {game.get('game_date', 'Unknown')}"
                        log_error(error_msg)
                        print(f"  WARNING: No player data found - logged to errors")
                    else:
                        # Accumulate results
                        table_counts = {}
                        for table_name, df in game_tables.items():
                            if not df.empty:
                                master_tables[table_name].append(df)
                                table_counts[table_name] = len(df)
                            else:
                                table_counts[table_name] = 0
                        
                        # Display table counts - more concise
                        total_records = sum(table_counts.values())
                        print(f"  Data: {total_records} total records ({', '.join([f'{k}: {v}' for k, v in table_counts.items()])})")
                    
                    successful_scrapes += 1
                    success_rate = (successful_scrapes / (successful_scrapes + failed_scrapes)) * 100
                    print(f"  SUCCESS ({successful_scrapes}/{successful_scrapes+failed_scrapes}) {success_rate:.1f}%")
                else:
                    # Both URLs failed - log error
                    error_msg = f"404 BOTH URLs - {game_id} | Primary: {url} | Alternate: {alt_url} | Teams: {game['away_team']} @ {game['home_team']} | Date: {game.get('game_date', 'Unknown')}"
                    log_error(error_msg)
                    print(f"  ERROR: Both URLs 404'd - logged to errors")
                    failed_scrapes += 1
                
            else:
                # Other HTTP error
                error_msg = f"HTTP {response.status_code} - {game_id} | {url}"
                log_error(error_msg)
                print(f"  ERROR: HTTP {response.status_code}")
                failed_scrapes += 1
                
        except Exception as e:
            error_msg = f"EXCEPTION - {game_id} | {url} | Error: {str(e)}"
            log_error(error_msg)
            print(f"  ERROR: {e}")
            failed_scrapes += 1
        
        # Calculate time remaining - simplified
        if i > 0:
            avg_time_per_game = elapsed.total_seconds() / (i + 1)
            estimated_remaining_seconds = games_remaining * avg_time_per_game
            estimated_completion = datetime.now() + timedelta(seconds=estimated_remaining_seconds)
            
            print(f"  ETA: {estimated_completion.strftime('%H:%M:%S')} ({format_time_remaining(estimated_remaining_seconds)} remaining)")
        
        # Random delay between requests with live countdown and random NFL fact
        if i < len(games_to_scrape) - 1:  # Don't delay after last game
            delay = random.randint(15, 45)
            
            # Show random NFL fact with subtle formatting
            random_fact = random.choice(nfl_facts)
            print(f"\nNFL FACT: {random_fact}\n")
            
            for countdown in range(delay, 0, -1):
                print(f"\r  Waiting {countdown} seconds...", end="", flush=True)
                time.sleep(1)
            print()  # New line after countdown
    
    # Combine and save all DataFrames
    total_time = datetime.now() - start_time
    print(f"\n=== COMBINING RESULTS ===")
    print(f"Total scraping time: {str(total_time).split('.')[0]}")
    
    final_tables = {}
    
    for table_name, df_list in master_tables.items():
        if df_list:
            combined_df = pd.concat(df_list, ignore_index=True)
            final_tables[table_name] = combined_df
            
            # Save to protected subdirectory
            filename = os.path.join(output_dir, f"2024_season_{table_name}.csv")
            combined_df.to_csv(filename, index=False)
            print(f"SAVED: {table_name}: {len(combined_df)} records -> {filename}")
        else:
            print(f"ERROR: {table_name}: No data collected")
    
    # Save backup copies with timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    backup_dir = os.path.join(output_dir, f"backup_{timestamp}")
    os.makedirs(backup_dir, exist_ok=True)
    
    for table_name, df in final_tables.items():
        backup_filename = os.path.join(backup_dir, f"2024_season_{table_name}_backup.csv")
        df.to_csv(backup_filename, index=False)
    
    print(f"BACKUP: All files backed up to {backup_dir}")
    
    # Final logging
    final_msg = f"SCRAPING COMPLETE - Success: {successful_scrapes} | Failed: {failed_scrapes} | Total time: {str(total_time).split('.')[0]}"
    log_error(final_msg)
    
    print(f"\nSCRAPING COMPLETE!")
    print(f"Successful: {successful_scrapes}")
    print(f"Failed: {failed_scrapes}")
    print(f"Success rate: {successful_scrapes/(successful_scrapes+failed_scrapes)*100:.1f}%")
    print(f"Total time: {str(total_time).split('.')[0]}")
    print(f"Errors logged to: scraper_errors.log")
    print(f"Data saved to: {output_dir}")
    
    return final_tables

# DANGER ZONE: Uncomment to run full season scrape
# batch_scrape_season()

In [ ]:
# REMOVED: Debug code for parser troubleshooting
# This was used during development but no longer needed